In [ ]:
import locale
# Set to German locale to get comma decimal separater
locale.setlocale(locale.LC_NUMERIC, "deu_DEU")

import numpy as np
import matplotlib.pyplot as plt
plt.rcdefaults()

# Tell matplotlib to use the locale we set above
plt.rcParams['axes.formatter.use_locale'] = True
%matplotlib qt5

figs, axes = plt.subplots(3,3, figsize=(9,6), sharex=True, sharey='row')



In [ ]:
locations
delays = []
for row_id in range(len(locations)):
    for column_id in range(len(locations[0])):
        file = open(location[row_id][column_id],'rb')
        data = pickle.load(file)
        data = data['S-parameter']
        unwrapped = np.unwrap(np.angle(data[2])).T
        delays.append((unwrapped[-1,:]-unwrapped[0,:]+np.pi*2)/(data[1][1][-1]-data[1][1][0]))
delay = np.median(delays)

In [ ]:
anticrossing_matrix = []
max_dev_matrix = []
coil_value_matrix = []

for row_id in range(len(locations)):
    anticrossing_row = []
    max_dev_row = []
    coil_value_row = []
    for column_id in range(len(locations[0])):
        data_nodelay = (data[2]*np.exp(-1j*delay*data[1][1])).T
        data_deviation_complex = data_nodelay - np.mean(data_nodelay, axis=0)
        axes[column_id, row_id].pcolormesh(data[1][0], data[1][1]/1e9, np.abs(data_deviation_complex), cmap='RdBu')
        max_dev = data[1][1][np.argmax(np.abs(data_deviation_complex), axis=0)]/1e9
        axes[column_id, row_id].plot(data[1][0], max_dev, color='yellow', linewidth=2)
        if column_id == 0:
            axes[column_id, row_id].set_title('Линия #'+str(row_id+1))
        if row_id == 0:
            axes[column_id, row_id].set_ylabel('Резонатор #'+str(column_id+1))
        anticrossing_row.append(data_deviation_complex)
        max_dev_row.append(max_dev)
        coil_value_row.append(data[1][0])
    anticrossing_matrix.append(anticrossing_row)
    max_dev_matrix.append(max_dev_row)
    coil_value_matrix.append(coil_value_row)
figs.text(0.5, 0.04, 'Напряжение управляющей линии, В', ha='center')
figs.text(0.02, 0.5, 'Частота, ГГц', va='center', rotation='vertical')
#plt.tight_layout()
print (delay/2/np.pi)b

In [ ]:
fqb = lambda x, EJ1, EJ2, EC, phi0, L: (8*EC)**0.5*((EJ1-EJ2)**2*np.sin(np.pi*x*L+phi0*np.pi)**2+(EJ1+EJ2)**2*np.cos(np.pi*x*L+phi0*np.pi)**2)**0.25
fr  = lambda x, frb, g, EJ1, EJ2, EC, phi0, L: (fqb(x, EJ1, EJ2, EC, phi0, L)+frb)*0.5+(((fqb(x, EJ1, EJ2, EC, phi0, L)-frb)*0.5)**2+g**2)**0.5*np.sign(frb-fqb(x, EJ1, EJ2, EC, phi0, L))

In [ ]:
initial_guess = {'L':np.identity(len(x))*0.3-0.01, # initial guess for inductance matrix
                     'phi0':-np.ones(len(x))*0.1, # initial guess for residual flux
                     'frb':np.mean(max_dev_matrix, axis=(0,2))*1e9,
                     #'frb':[6.85e9, 7.15e9, 7.00e9],
                     'EC':0.175e9,
                     'EJ1':18.8e9,
                     'EJ2':3.6e9,
                     'g':40e6}

num_resonators =3
phi0_vec = []
L_mat = []
fitresults_vec = []
for resonator_id in range(num_resonators):
    data = [max_dev_matrix[i][resonator_id] for i in range(num_resonators)]
    x = [coil_value_matrix[i][resonator_id] for i in range(num_resonators)]
    def fr_coil(p, coil_id):
        frb, g, EJ1, EJ2, EC, phi0 = p[:6]
        L = p[6:]
        return fr(x[coil_id], frb, g, EJ1, EJ2, EC, phi0, L[coil_id])/1e9
    def residuals(p):
        frb, g, EJ1, EJ2, EC, phi0 = p[:6]
        L = p[6:]
        res = []
        for coil_id in range(len(data)):
            #print (fr(x[coil_id], frb, g, EJ1, EJ2, EC, phi0, L[coil_id]))
            #print (fqb(x[coil_id], EJ1, EJ2, EC, phi0, L[coil_id]))
            #print (x[coil_id]*np.reshape(L,(-1,1)))
            res.extend(fr(x[coil_id], frb, g, EJ1, EJ2, EC, phi0, L[coil_id])/1e9-data[coil_id])
        return res
    from scipy.optimize import least_squares

    p0 = (initial_guess['frb'][resonator_id], 
          initial_guess['g'], 
          initial_guess['EJ1'], 
          initial_guess['EJ2'], 
          initial_guess['EC'], 
          initial_guess['phi0'][resonator_id])+\
          tuple(initial_guess['L'][resonator_id,:])

    bounds = ((0,      0,      0,      0,      0,      -np.pi)+tuple([-np.inf]*num_resonators),
              (np.inf, np.inf, np.inf, np.inf, np.inf,  np.pi)+tuple([ np.inf]*num_resonators))
    fitresults = least_squares(residuals, p0, x_scale=np.abs(p0), bounds=bounds)
    fitresults_vec.append(fitresults)
    phi0_vec.append(fitresults.x[5])
    L_mat.append(fitresults.x[6:])

In [ ]:
for resonator_id in range(num_resonators):
    for coil_id in range(num_resonators):
        axes[resonator_id, coil_id].plot(coil_value_matrix[coil_id][resonator_id], 
                                     fr_coil(fitresults_vec[resonator_id].x, coil_id), color='black', linewidth=2)